In [1]:
import numpy as np
import pandas as pd
# thư viện này để tạo ma trận thưa 
from scipy.sparse import csr_matrix
# thư viện này dùng để xây dựng mô hình knn
from sklearn.decomposition import TruncatedSVD

## đọc dữ liệu


In [2]:
column_names1 = ['user id','movie id','rating','timestamp']
dataset = pd.read_csv('Data Movielens/u.data', sep='\t',header=None,names=column_names1)
d = 'movie id | movie title | release date | video release date | IMDb URL | unknown | Action | Adventure | Animation | Children | Comedy | Crime | Documentary | Drama | Fantasy | Film-Noir | Horror | Musical | Mystery | Romance | Sci-Fi | Thriller | War | Western'
column_names2 = d.split(' | ')
items_dataset = pd.read_csv('Data Movielens/u.item', sep='|',header=None,names=column_names2,encoding='latin-1')
movie_dataset = items_dataset[['movie id','movie title']]

In [4]:
items_dataset

,movie id,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Children,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1677,1678,Mat' i syn (1997),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?Mat%27+i+syn+...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1678,1679,B. Monkey (1998),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?B%2E+Monkey+(...,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
1679,1680,Sliding Doors (1998),01-Jan-1998,NaN,http://us.imdb.com/Title?Sliding+Doors+(1998),0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1680,1681,You So Crazy (1994),01-Jan-1994,NaN,http://us.imdb.com/M/title-exact?You%20So%20Cr...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0



# tiền xử lý dữ liệu

In [5]:
# B1: hợp nhất 2 dataframe dataset và movie_dataset đã đọc ở trên dựa trên cột chung là movie id
merged_dataset = pd.merge(dataset, movie_dataset, how='inner', on='movie id')

# B2: sử dụng groupby và agg để tính điểm trung bình đánh giá của mỗi người dùng cho mỗi bộ phim.
refined_dataset = merged_dataset.groupby(by=['user id','movie title'], as_index=False).agg({"rating":"mean"})

# B3: Tạo một DataFrame mới chứa số lượng đánh giá cho mỗi điểm đánh giá.
rating_count_df = pd.DataFrame(refined_dataset.groupby(['rating']).size(), columns=['count'])


In [6]:
merged_dataset

,user id,movie id,rating,timestamp,movie title
0,196,242,3,881250949,Kolya (1996)
1,63,242,3,875747190,Kolya (1996)
2,226,242,5,883888671,Kolya (1996)
3,154,242,3,879138235,Kolya (1996)
4,306,242,5,876503793,Kolya (1996)
...,...,...,...,...,...
99995,840,1674,4,891211682,Mamma Roma (1962)
99996,655,1640,3,888474646,"Eighth Day, The (1996)"
99997,655,1637,3,888984255,Girls Town (1996)
99998,655,1630,3,887428735,"Silence of the Palace, The (Saimt el Qusur) (1..."


In [7]:
rating_count_df

,count
rating,
1.0,6083
1.5,3
2.0,11334
2.5,6
3.0,27060
3.5,19
4.0,34042
4.5,16
5.0,21130


In [8]:
# B4: Trong bộ dữ liệu sẽ có các đánh giá bị bỏ trống 
# (người dùng không đánh giá phim này), ta sẽ tính số lượng 
# các đánh giá thiếu này và bổ sung vào rating_count_df
num_users = len(refined_dataset['user id'].value_counts())
num_items = len(refined_dataset['movie title'].value_counts())
total_count = num_items * num_users
zero_count = total_count-refined_dataset.shape[0]
rating_count_df = rating_count_df.append(
    pd.DataFrame({'count': zero_count}, index=[0.0]),
    verify_integrity=True,
).sort_index()

# vì giá trị của số lượng đánh giá lớn, do đó ta giảm đi bằng cách lấy logarit của nó
rating_count_df['log_count'] = np.log(rating_count_df['count'])
rating_count_df = rating_count_df.reset_index().rename(columns={'index': 'rating score'})

C:\Users\LAPTOP\AppData\Local\Temp\ipykernel_19500\986140195.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  rating_count_df = rating_count_df.append(


In [9]:
rating_count_df

,rating score,count,log_count
0,0.0,1469459,14.200405
1,1.0,6083,8.713253
2,1.5,3,1.098612
3,2.0,11334,9.335562
4,2.5,6,1.791759
5,3.0,27060,10.205812
6,3.5,19,2.944439
7,4.0,34042,10.435350
8,4.5,16,2.772589
9,5.0,21130,9.958449


In [16]:
movie_to_user_df = refined_dataset.pivot(
     index='movie title',
     columns='user id',
      values='rating').fillna(0)

#Tạo một ma trận bảng đánh giá của người dùng cho các bộ phim, với các giá trị là điểm đánh giá.
# Chuyển đổi ma trận thành dạng ma trận thưa thớt để giảm lượng dữ liệu không cần thiết.
movie_to_user_sparse_df = csr_matrix(movie_to_user_df.values)
movies_list = list(movie_to_user_df.index)

# Tạo một từ điển với tên phim là khóa và chỉ số của nó từ movies_list:
movie_dict = {movie : index for index, movie in enumerate(movies_list)}
case_insensitive_movies_list = [i.lower() for i in movies_list]

In [17]:
movie_to_user_df

user id,1,2,3,4,5,6,7,8,9,10,...,934,935,936,937,938,939,940,941,942,943
movie title,,,,,,,,,,,,,,,,,,,,,
'Til There Was You (1997),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1-900 (1994),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101 Dalmatians (1996),2.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,0.0,2.0,4.0,0.0,0.0,0.0,0.0,0.0
12 Angry Men (1957),5.0,0.0,0.0,0.0,0.0,4.0,4.0,0.0,0.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
187 (1997),0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Young Guns II (1990),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
"Young Poisoner's Handbook, The (1995)",0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,...,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zeus and Roxanne (1997),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Xây dựng mô hình

## Điều chỉnh các tham số của thuật toán với <font color=red>GridSearchCV</font> để tìm ra các tham số tốt nhất cho thuật toán

In [8]:
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import GridSearchCV
import numpy as np

# Xác định lưới tham số
param_grid = {
    'n_components': [10, 20, 30, 40, 50],
    'n_iter': [5, 10, 15, 20, 25],
    'random_state': [42]  # You can add more values if needed
}

# Tạo mô hình TruncatedSVD
svd_model = TruncatedSVD()

# Tạo đối tượng GridSearchCV với phương sai được giải thích làm thước đo tính điểm
grid_search = GridSearchCV(svd_model, param_grid, scoring='explained_variance', cv=5, n_jobs=-1)

# Điều chỉnh tìm kiếm dạng lưới cho phù hợp với dữ liệu
grid_search.fit(movie_to_user_df)

#In các thông số tốt nhất được tìm thấy
print("Best Parameters: ", grid_search.best_params_)


Best Parameters:  {'n_components': 10, 'n_iter': 5, 'random_state': 42}


c:\Users\LAPTOP\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_search.py:979: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan]
  warnings.warn(


In [9]:
my_n_components = grid_search.best_params_['n_components']
my_n_iters = grid_search.best_params_['n_iter']
my_random_state = grid_search.best_params_['random_state']

## Sử dụng thư viện scikit-learn để tạo một mô hình SVD dựa trên ma trận thưa thớt.
## Mô hình này sẽ được sử dụng để tìm các bộ phim tương tự dựa trên độ tương đồng cosine giữa chúng.


In [10]:
svd_model = TruncatedSVD(n_components=my_n_components,n_iter = my_n_iters, random_state=my_random_state)
svd_matrix = svd_model.fit_transform(movie_to_user_df)

In [11]:
# hàm này lấy các chuỗi kí tự trong tên phim được nhập vào, 
# so sánh với danh sách phim 
# để tìm các phim mà có thể bạn  đang tìm kiếm
def get_possible_movies(movie):

    temp = ''
    possible_movies = case_insensitive_movies_list.copy()
    for i in movie :
      out = []
      temp += i
      for j in possible_movies:
        if temp in j:
          out.append(j)
      if len(out) == 0:
          return possible_movies
      out.sort()
      possible_movies = out.copy()

    return possible_movies

In [12]:
svd_matrix[0]

array([ 1.03999361,  0.65990124,  0.04568503,  0.81399544,  0.16695821,
       -0.98329431, -0.3227807 ,  0.309112  , -0.03848844, -0.73597038])

In [13]:

def get_svd_similar_movies(movie_name, num_recom):
    movie_idx = movie_dict[movie_name]
    movie_vector = svd_matrix[movie_idx].reshape(1, -1)
    
    # Tính toán độ tương đồng cosin
    cosine_similarities = np.dot(movie_vector, svd_matrix.T).flatten()
    
    # Min-Max normalization
    min_val = np.min(cosine_similarities)
    max_val = np.max(cosine_similarities)
    
    #Chuẩn hóa thành phạm vi [0, 1]
    normalized_similarities = (cosine_similarities - min_val) / (max_val - min_val)
    
    # Nhận chỉ số của hầu hết các bộ phim tương tự
    similar_movies_indices = np.argsort(normalized_similarities)[::-1][1:num_recom+1]
    
    # Tạo đề xuất có điểm tương đồng được chuẩn hóa
    recommendations = [
        f"{movies_list[i]} (Similarity: {normalized_similarities[i]:.2f})"
        for i in similar_movies_indices
    ]
    
    return recommendations

In [14]:
# hàm trả về tên các bộ phim đề xuất dựa vào từ khóa nhập vào
def recommend():
    try:
        movie_name = entry_movie.get()
        movie_name_lower = movie_name.lower()
        if movie_name_lower not in case_insensitive_movies_list:
            raise InvalidInput
        else:
            num_recom = int(entry_num_recom)
            recommendations = get_svd_similar_movies(movies_list[case_insensitive_movies_list.index(movie_name_lower)], num_recom)
            result_label.config(text="\n".join(recommendations))
    except InvalidInput:
        possible_movies = get_possible_movies(movie_name_lower)
        if len(possible_movies) == len(movies_list):
            result_label.config(text="Movie name entered does not exist in the list.")
        else:
            indices = [case_insensitive_movies_list.index(i) for i in possible_movies]
            suggestions = '\n'.join([movies_list[i] for i in indices])
            result_label.config(text=suggestions)

# Tạo giao diện người dùng

In [15]:
import tkinter as tk
class InvalidInput(Exception):
    pass
# # định nghĩa số lượng phim đề xuất     
entry_num_recom = 10
root = tk.Tk()
root.title("SVD Movie Recommendation System")

label_movie = tk.Label(root, text="Enter the Movie name:")
label_movie.pack(pady=10)

entry_movie = tk.Entry(root, width=30)
entry_movie.pack(pady=10)


button_recommend = tk.Button(root, text="Search", command=recommend)
button_recommend.pack(pady=20)

result_label = tk.Label(root, text="")
result_label.pack(pady=10)
# Run the Tkinter event loop
root.mainloop()